In [ ]:
import os
import re
import pandas as pd
import subprocess

def izdvoji_naziv_proteina(zaglavlje):
    """
    Izdvaja naziv proteina bez sadržaja u uglastim zagradama.
    """
    podudaranje = re.match(r"^(\S+)\s+(.*)", zaglavlje)
    if podudaranje:
        id_proteina = podudaranje.group(1)
        opis_proteina = podudaranje.group(2)
        # Uklanjamo sadržaj unutar uglastih zagrada (ako postoji)
        opis_proteina = re.sub(r"\[.*?\]", "", opis_proteina).strip()
        return f"{id_proteina} __{opis_proteina}"
    return zaglavlje.strip()


def podeli_fasta_fajl(ulazni_fajl, izlazni_direktorijum):
    """
    Podeli FASTA fajl na pojedinačne fajlove sa nazivima proteina iz zaglavlja.
    """
    if not os.path.exists(izlazni_direktorijum):
        os.makedirs(izlazni_direktorijum)

    with open(ulazni_fajl, 'r') as fajl:
        sekvenca = ""
        zaglavlje = ""
        imena_fajlova = set()

        for linija in fajl:
            linija = linija.strip()
            if linija.startswith(">"):
                if sekvenca:
                    naziv_fajla = izdvoji_naziv_proteina(zaglavlje)
                    naziv_fajla = re.sub(r"[\\/:*?\"<>|]", "_", naziv_fajla)  # Zamenjuje neprikladne znakove
                    if naziv_fajla not in imena_fajlova:
                        imena_fajlova.add(naziv_fajla)
                        putanja = os.path.join(izlazni_direktorijum, f"{naziv_fajla}.fasta")
                        with open(putanja, 'w') as izlazni_fajl:
                            izlazni_fajl.write(f">{zaglavlje}\n{sekvenca}\n")
                    sekvenca = ""
                zaglavlje = linija[1:]
            else:
                sekvenca += linija

        if sekvenca:
            naziv_fajla = izdvoji_naziv_proteina(zaglavlje)
            naziv_fajla = re.sub(r"[\\/:*?\"<>|]", "_", naziv_fajla)
            if naziv_fajla not in imena_fajlova:
                imena_fajlova.add(naziv_fajla)
                putanja = os.path.join(izlazni_direktorijum, f"{naziv_fajla}.fasta")
                with open(putanja, 'w') as izlazni_fajl:
                    izlazni_fajl.write(f">{zaglavlje}\n{sekvenca}\n")

def pokreni_isunstruct_na_fajlovima(izlazni_direktorijum):
    """
    Pokreće IsUnstruct za svaki FASTA fajl u datom direktorijumu.
    """
    rezultati = []
    fasta_fajlovi = sorted([f for f in os.listdir(izlazni_direktorijum) if f.endswith('.fasta')])

    for i, fasta_fajl in enumerate(fasta_fajlovi, 1):
        putanja_fajla = os.path.join(izlazni_direktorijum, fasta_fajl)
        print(f"Obrada {i}/{len(fasta_fajlovi)}: {fasta_fajl}")

        try:
            rezultat = subprocess.run(
                ["/home/user/Desktop/IP2/IsUnstruct_2.02/IsUnstruct", "-use_pattern", "2", "-long_disp", "1", "-file_save", "1", putanja_fajla],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )
            if rezultat.stderr:
                print(f"Upozorenje: {rezultat.stderr.strip()}")
            rezultati.append((fasta_fajl, rezultat.stdout))
        except Exception as greska:
            print(f"Greška pri obradi {fasta_fajl}: {greska}")

    return rezultati

def parsiraj_isunstruct_izlaz(izlaz_isunstruct):
    linije = izlaz_isunstruct.splitlines()
    rezultati = []

    if linije[0].startswith("# IsUnstruct") and "long format" in linije[0]:
        rezultati = parsiraj_dugacki_format(linije)
    elif linije[0].startswith("# IsUnstruct") and "short format" in linije[0]:
        rezultati = parsiraj_kratki_format(linije)
    else:
        raise ValueError("Nepoznat format izlaza.")

    return rezultati

def kreiraj_podatke_iz_rezultata(rezultati):
    """
    Kreira DataFrame sa informacijama o sekvencama, uključujući pozicije, 
    aminokiseline, skorove i klasifikacije.
    """
    redovi = []
    for ime_fajla, izlaz in rezultati:
        try:
            parsirani_rezultati = parsiraj_isunstruct_izlaz(izlaz)
            print(f"Broj redova za {ime_fajla}: {len(parsirani_rezultati)}")
            for rezultat in parsirani_rezultati:
                redovi.append({
                    "Naziv proteina": ime_fajla.replace(".fasta", ""),
                    "Pozicija": rezultat["Pozicija"],
                    "Aminokiselina": rezultat["Aminokiselina"],
                    "Verovatnoća": rezultat["Verovatnoća"],
                    "Klasifikacija": rezultat["Klasifikacija"]
                })
        except Exception as greska:
            print(f"Greška pri obradi izlaza za {ime_fajla}: {greska}")

    print(f"Ukupan broj redova u DataFrame-u: {len(redovi)}")
    df = pd.DataFrame(redovi)
    
    
    def prioritet_grupisanja(naziv):
        naziv = naziv.lower()
        if "nucleoprotein" in naziv or "nucleocapsid" in naziv:
            return 1
        elif "spike" in naziv:
            return 2
        elif "matrix" in naziv:
            return 3

    df["Grupa"] = df["Naziv proteina"].apply(prioritet_grupisanja)
    df["Originalni redosled"] = range(len(df))  # Zadržavanje izvornog redosleda
    df.sort_values(by=["Grupa", "Originalni redosled"], inplace=True)
    df.drop(columns=["Grupa", "Originalni redosled"], inplace=True)  # Uklanjanje pomoćnih kolona

    return df

def parsiraj_dugacki_format(lines):
    results = []
    for line in lines:
        line = line.strip()
        if line and not line.startswith("#"):  # Ignoriši komentare
            parts = line.split()
            try:
                # Ako linija sadrži tačno 4 vrednosti
                if len(parts) == 4:
                    position = int(parts[0])  # Pozicija
                    amino_acid = parts[1]  # Aminokiselina
                    state = parts[2]  # Stanje (U ili P)
                    probability = float(parts[3])  # Verovatnoća

                    classification = "neuređena" if probability > 0.5 else "uređena"
                    
                    results.append({
                        "Pozicija": position,
                        "Aminokiselina": amino_acid,
                        "Stanje": state,
                        "Verovatnoća": probability,
                        "Klasifikacija": classification
                    })
                elif len(parts) > 4:
                    position = int(parts[0])  # Pozicija
                    amino_acid = parts[1]  # Aminokiselina
                    state = parts[3]  # Stanje (U ili P)
                    probability = float(parts[4])  # Verovatnoća

                    # Ispisujemo dodatne vrednosti koje ignoriramo
                    if len(parts) > 4:
                        print(f"Ignorisano: {parts[2:]}")  # Dodaj ispis da vidiš šta je preostalo
                        
                    classification = "neuređena" if probability > 0.5 else "uređena"
                    
                    results.append({
                        "Pozicija": position,
                        "Aminokiselina": amino_acid,
                        "Stanje": state,
                        "Verovatnoća": probability,
                        "Klasifikacija": classification
                    })
                else:
                    print(f"Warning: Ignoring invalid line: {line}")
            except ValueError:
                print(f"Invalid line ignored: {line}")
    return results    

def parsiraj_kratki_format(linije):
    rezultati = []
    sekvenca, stanje, verovatnoce = "", "", ""

    for linija in linije:
        if linija.startswith("sequence"):
            sekvenca = linija.split()[1]
        elif linija.startswith("state"):
            stanje = linija.split()[1]
        elif linija.startswith("probability"):
            verovatnoce = linija.split()[1]
        elif linija.strip() == "" and sekvenca:
            for i, (aa, s, prob) in enumerate(zip(sekvenca, stanje, verovatnoce)):
                rezultati.append({
                    "Pozicija": i + 1,
                    "Aminokiselina": aa,
                    "Verovatnoća": float(prob),
                    "Klasifikacija": "neuređena" if s == "U" else "uređena"
                })
            sekvenca, stanje, verovatnoce = "", "", ""

    return rezultati

def glavna_funkcija():
    ulazni_fajl = "/home/user/Desktop/IP2/izdvojeno_proteini_ebola.fasta"
    izlazni_direktorijum = "/home/user/Desktop/IP2/IsUnstruct_2.02/___split__sekvence_"

    podeli_fasta_fajl(ulazni_fajl, izlazni_direktorijum)
    rezultati = pokreni_isunstruct_na_fajlovima(izlazni_direktorijum)
    df = kreiraj_podatke_iz_rezultata(rezultati)

    df.to_csv("/home/user/Desktop/IP2/IsUnstruct_2.02/_rezultati__isunstruct__EBOLA_.csv", index=False)
    print("Rezultati su uspešno sačuvani u CSV fajl.")

# Pokretanje glavne funkcije
glavna_funkcija() 